In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import datetime as dt

import pandas as pd

from deep_hedging import (
    Ticker,
    Underlyings,
    Frequency,
    YieldCurves,
    NelsonSiegelCurve,
    ConstantRateCurve,
    FixedCouponBond,
)
from deep_hedging.non_linear.exotic.basket import WorstOfDigitalMemoryCall
from deep_hedging.non_linear.exotic.quanto import QuantoOption

RANDOM_SEED = 12
PATH = Path("../../../data/")

## Define product parameters.

In [3]:
FIXED_PAYMENT = 0.005
DIGITAL_COUPON = 5.0
PAYMENT_BARRIER = 1.0
PAYMENT_FREQUENCY = Frequency.SEMIANNUALLY

FIXED_FINAL_PAYMENT = 1.0

N_YEARS_LOOKBACK = 3
FIXING = dt.datetime(2021, 4, 19)
FINAL_FIXING = dt.datetime(2024, 4, 17)

EUR_RATE = -0.197 / 100
GBP_RATE = 0.6045 / 100

## Get market data.

In [4]:
underlyings = Underlyings(
    tickers=[
        Ticker("Sanofi", "SAN.PA", currency="EUR"),
        Ticker("GlaxoSmithKline PLC", "GSK.L", currency="GBP"),
        Ticker("Bayer AG", "BAYN.DE", currency="EUR"),
    ],
    start=N_YEARS_LOOKBACK,
    end=FIXING,
)

[*********************100%%**********************]  3 of 3 completed


In [5]:
fx = Underlyings(
    tickers=[
        Ticker("EURRUB", "EURRUB=X", currency="EUR"),
        Ticker("GBPRUB", "GBPRUB=X", currency="GBP"),
    ],
    start=N_YEARS_LOOKBACK,
    end=FIXING,
)

# Issue with yahoo finance data - strange GBPRUB quote
fx.data = fx.data[fx["GBPRUB=X"] > 1.2]
fx.data

[*********************100%%**********************]  2 of 2 completed


Ticker,EURRUB=X,GBPRUB=X
Date,,
2004-02-13,36.470001,53.591999
2004-04-01,34.990002,52.884998
2004-07-27,35.070000,52.995998
2005-01-17,36.619999,52.074001
2005-02-22,36.689999,53.369999
...,...,...
2021-04-12,92.128601,106.120071
2021-04-13,92.092499,106.289673
2021-04-14,90.795601,104.455833


In [6]:
nss_params = pd.read_csv(PATH / "rub_nss.csv", sep=";")
rub_params = nss_params[
    nss_params["tradedate"] == f"{FIXING.strftime('%d.%m.%Y')}"
].loc[:, ["B1", "B2", "B3", "T1"]]
rub_params = rub_params.replace(",", ".", regex=True).astype(float)

rub_curve = NelsonSiegelCurve(
    currency="RUB",
    b0=rub_params.B1.values[0] / 100,
    b1=rub_params.B2.values[0] / 100,
    b2=rub_params.B3.values[0] / 100,
    tau=rub_params.T1.values[0] / 100,
)

In [7]:
curves = YieldCurves(
    [
        rub_curve,
        ConstantRateCurve(
            currency="EUR",
            constant_rate=EUR_RATE,  # as no NSS params are available, using flat curve
        ),
        ConstantRateCurve(
            currency="GBP",
            constant_rate=GBP_RATE,  # as no NSS params are available, using flat curve
        ),
    ]
)

## Run pricing.

In [8]:
fixed_bond = FixedCouponBond(
    yield_curve=rub_curve,
    start_date=FIXING,
    end_date=FINAL_FIXING,
    fixed_coupon=FIXED_PAYMENT,
    frequency=PAYMENT_FREQUENCY,
)

In [9]:
worst_of_digital = WorstOfDigitalMemoryCall(
    underlyings=underlyings,
    yield_curve=rub_curve,
    strike_level=PAYMENT_BARRIER,
    frequency=PAYMENT_FREQUENCY,
    start_date=FIXING,
    end_date=FINAL_FIXING,
    digital_coupon=DIGITAL_COUPON,
    random_seed=RANDOM_SEED,
)

quanto = QuantoOption(
    option=worst_of_digital,
    modifying_underlyings=fx,
    yield_curves=curves,
    random_seed=RANDOM_SEED,
)

warrant = fixed_bond + quanto
print(f"{warrant.price():,.4f} RUB")

9.5099 RUB
